### Description

This notebook demonstrates how to create a circuit for simulating unitary dynamics of a 1-D Heizenberg spin-1/2 chain with the nearest-neighbour interactions described by the following Hamiltonian:

$$
H_{HZ} = -\sum_{\alpha} J_{\alpha} \sum_{i = 1}^{N - 1} \sigma_i^{\alpha} \otimes \sigma_{i+1}^{\alpha},
$$

where $\alpha \in \{X, Y, Z\}$ and $\sigma_i^{\alpha}$ denote Pauli matrices acting on i-th spin. Here, I implemented the evolution operator $U_{HZ} = e^{-itH_{HZ}}$ using an n-step (first-order)Trotter  decomposition. Since not all terms in $H_{HZ}$ commute, different factorizations of $U_{HZ}$ are possible based on how mutually commuting terms are grouped (different groupings lead to different approximation errors). One possibility is to split $H_{HZ}$ into a sum of pariwise interactions and define an evolution operator for each pair. The pair propagators can be written as products of exponentials

$$
U_{i, i+1} = \prod_{\alpha} e^{itJ_{\alpha}\sigma_i^{\alpha} \otimes \sigma_{i+1}^{\alpha}}
$$

and implemented as a circuit with at most 3 CNOT gates [https://arxiv.org/pdf/quant-ph/0308006]. For the sake of time, I chose a sub-optimal implementation which uses 4 CNOTs which is still better than a straightforward textbook method (it would require 6 CNOTs). The pair blocks are then arranged into layers as shown in the diagram to complete circuit construction:
<div style="text-align: center;">
  <img src="circuit_diagram.png" alt="Trotter Ciruict">
</div>

(orange blocks correspond to spin pairs (0,1), (2, 3), etc. Blue blocks correspond the complementary set of pairs).

### Comments

- Alternative Trotter schemes are possible. For example, one could group the terms corresponding to the same spin projection. In a supplementary notebook I show that such scheme is expected to give a slightly larger Trotter error compared to the one described above. 
- Using symmetry properties (Yang-Baxter equation) of the Trotter circuits they can be further compressed to enable **constant depth simulation of 1-D Heizenberg chains**. Further details can be found in Gulania et al. [https://arxiv.org/pdf/2112.01690] and Bassman et al. [https://arxiv.org/pdf/2103.07429]. An implementation of the circuit compression: https://github.com/ZichangHe/QuYBE [https://arxiv.org/abs/2212.06948]

In [15]:
import cirq
import numpy as np
from circuits import UnitaryEvolution2Site
from typing import List

In [13]:
#help(cirq.LineQubit)
#help(cirq.GridQubit)

In [18]:
def create_trotter_circuit(j: np.array, total_time: float, n_trotter: int, qubits: List[cirq.LineQubit]) -> cirq.Circuit:
    '''Create a circuit for simulating time evolution of a 1-D Heizenberg chain using Trotter approximation
    for the propagator. 
    '''
    
    circuit = cirq.Circuit()
    dt = total_time / n_trotter # time step for a single Trotter factor
    n_qubits = len(qubits)
    angles = j * dt
    
    for _  in range(n_trotter):
        first_sublayer = cirq.Moment([
            UnitaryEvolution2Site(angles).on(qubits[i], qubits[i+1]) for i in range(0, n_qubits - 1, 2)
            ])
        second_sublayer = cirq.Moment([
            UnitaryEvolution2Site(angles).on(qubits[i], qubits[i+1]) for i in range(1, n_qubits - 1, 2)
            ])
        circuit.append([first_sublayer, second_sublayer])
        
    return circuit

In [21]:
# Demo
qubits = cirq.LineQubit.range(6)
couplings = np.array([1., 2., -1.5])
evolution_time = 1.5
n_trotter_steps = 2

trotter_circuit = create_trotter_circuit(j=couplings, total_time=evolution_time, n_trotter=n_trotter_steps, qubits=qubits)

In [22]:
print(trotter_circuit)

0: ───Uh────────Uh────────
      │         │
1: ───Uh───Uh───Uh───Uh───
           │         │
2: ───Uh───Uh───Uh───Uh───
      │         │
3: ───Uh───Uh───Uh───Uh───
           │         │
4: ───Uh───Uh───Uh───Uh───
      │         │
5: ───Uh────────Uh────────
